In [2]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import gc
import os
# import numpy as np
from skimage import io, color, filters, measure, exposure, feature
from shapely.geometry import MultiPoint, Polygon
import shapely
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.collections import PatchCollection
import numpy as np


In [3]:

#reproduce cells
def reproduce(occupations, active_cell_counts, age_of_first_occupations, t, alpha, dT, gamma, delta, beta, senesc_rate, prob_diff):
    for n in range(N): #all of these should have occupation number 1 or 0 after the last adjustment step
        if occupations[n] == 1:
            if active_cell_counts[n] == 1: #is the cell able to actively reproduce?
                if np.random.rand() < senesc_rate*dT:
                    active_cell_counts[n] = 0 #render the cell inactive if we're trialling senescence
                else:
                    #where is occupied?
                    dists_to_other_cells = dists_between_positions[n][np.where(occupations==1)[0]]
                    dists_to_other_cells = dists_to_other_cells[np.nonzero(dists_to_other_cells)[0]]#n records distances to all other cells, except itself
                    prob_rep = alpha - np.nansum(beta/dists_to_other_cells) if len(dists_to_other_cells) > 0 else alpha
                    if np.random.rand() < prob_rep*dT: #compute division probability
                        occupations[n] = 2 #cell is double-occupied
                        if np.random.rand() < prob_diff: #is this new cell active?
                            active_cell_counts[n] = 2 
    return occupations, active_cell_counts, age_of_first_occupations


#adjust positions until we have no double occupations
#rule: if a cell becomes double-occupied through, then mark which cell was just added to it
def shift(occupations, active_cell_counts, age_of_first_occupations, type_just_added, t, alpha, dT, gamma, delta, beta, senesc_rate, prob_diff):
    keep_adjusting = True
    while keep_adjusting: 
        any_double_occupied = False #assume none are double occupied
        for n in range(N): #run through every point in space
            if occupations[n] > 1: #is it double-occupied? 
                any_double_occupied = True
                empty_neighbours = list(filter(lambda i: (occupations[i]==0), neighbours[n])) #are there any empty neighbours?
                if len(empty_neighbours)==0: #if all neighbours are occupied
                    ps = [np.exp(-gamma*occupations[i]) for i in neighbours[n]] #weight by their occupations
                    norm = sum(ps) #normalise
                    ps = [p/norm for p in ps]
                    index = np.random.choice(neighbours[n], size=1, p = ps)[0] #choose one to move to at random
                    occupations[index] += 1 #move it to that lattice-point
                    #are we moving an active cell or an inactive cell?
                    if type_just_added[n] == 1: #active cell was just moved there so choose from between other types
                        prob_active = (active_cell_counts[n]-1)/(occupations[n]-1) #fraction of the remaining cells which are active
                        if np.random.rand() < prob_active:
                            active_cell_counts[n] -= 1 #it is active
                            active_cell_counts[index] += 1 #so move it
                            type_just_added[index] = 1 #type just moved was active
                        else:
                            type_just_added[index] = -1 #we just moved an inactive cell
                    else:
                        if type_just_added[n] == -1:
                            prob_active = (active_cell_counts[n])/(occupations[n]-1) #fraction of the remaining cells which are active (retain an inactive cell)
                            if np.random.rand() < prob_active:
                                active_cell_counts[n] -= 1 #it is active
                                active_cell_counts[index] += 1 #so move it
                                type_just_added[index] = 1 #we just moved an active cell here
                            else:
                                type_just_added[index] = -1 #we just moved an inactive cell here
                        else: #we have no information about what was just added
                            prob_active = active_cell_counts[n]/occupations[n] #what fraction of these cells are active?
                            if np.random.rand() < prob_active:
                                active_cell_counts[n] -= 1 #it is active
                                active_cell_counts[index] += 1 #so move it
                                type_just_added[index] = 1 
                            else:
                                type_just_added[index] = -1
                else:
                    contact_list = [len(list(filter(lambda i: (occupations[i]>0), neighbours[m]))) for m in empty_neighbours] #number of neighbours possessed by every neighbouring empty lattice-point
                    weights = [np.exp(delta*c) for c in contact_list] #weight by number of neighbours
                    norm = sum(weights)
                    ps = [w/norm for w in weights]
                    index = np.random.choice(empty_neighbours, size=1, p=ps)[0]
                    occupations[index] += 1 #move to an empty one at random
                    age_of_first_occupations[index] = t
                    if type_just_added[n] == 1: #active cell was just moved there so choose from between other types
                        prob_active = (active_cell_counts[n]-1)/(occupations[n]-1)
                        if np.random.rand() < prob_active:
                            active_cell_counts[n] -= 1 #it is active
                            active_cell_counts[index] += 1 #so move it
                            type_just_added[index] = 1
                        else:
                            type_just_added[index] = -1 #moving an inactive cell
                    else:
                        if type_just_added[n] == -1:
                            prob_active = (active_cell_counts[n])/(occupations[n]-1)
                            if np.random.rand() < prob_active:
                                active_cell_counts[n] -= 1 #it is active
                                active_cell_counts[index] += 1 #so move it
                                type_just_added[index] = 1
                            else:
                                type_just_added[index] = -1
                        else: #no information about what was just added
                            prob_active = active_cell_counts[n]/occupations[n]
                            if np.random.rand() < prob_active:
                                active_cell_counts[n] -= 1 #it is active
                                active_cell_counts[index] += 1 #so move it
                                type_just_added[index] = 1
                            else:
                                type_just_added[index] = -1
                 #   print("jammed")
                occupations[n] -= 1 #take a cell away from the original cell
        keep_adjusting = any_double_occupied #have we found any double-occupied?
    return occupations, active_cell_counts, type_just_added, age_of_first_occupations

In [4]:
#now run this once
#choose centre roughly

from scipy.spatial.distance import cdist

n = 30 #number of cells in each direction
N = n**3 #number of cells total


v_0 = 80 #volume of one cell
r_0 = np.cbrt(v_0)


xs, ys, zs = np.zeros(N), np.zeros(N), np.zeros(N)

overall_neighbour_list = []
positions = []


#make a list of neighbours
for i in range(n):
    for j in range(n):
        for k in range(n):
            neighbour_list = []
            index = i*(n**2) + j*n + k
            positions.append(np.array([i*r_0, j*r_0, k*r_0])) 
            xs[index], ys[index], zs[index] = i*r_0, j*r_0, k*r_0
            for i_ in range(i-1, i+2):
                for j_ in range(j-1, j+2):
                    for k_ in range(k-1, k+2):
                        new_index = i_*(n**2) + j_*n + k_
                        if (0 <= i_ < n) and (0 <= j_ < n) and (0 <= k_ < n): 
                            if 0 <= new_index < N and new_index != index:
                                neighbour_list.append(new_index)
            overall_neighbour_list.append(np.array(neighbour_list))
            
neighbours= overall_neighbour_list
dists_between_positions = cdist(positions, positions)

def run_simulation(alpha=0.0, dT=0.01, gamma=10, delta=1.0, beta=0.0, senesc_rate=0.0, prob_diff=1, max_cells=5000):
    t = 0

    age_of_first_occupations = np.zeros(N) #keep track of the time at which a lattice point was first occupiedimport numpy as np

    centre = (n**2)*int(n/2) + (n*int(n/2)) + int(n/2) #set 

    occupations, active_cell_counts, type_just_added = np.zeros(N), np.zeros(N), np.zeros(N) #set things up- one occupied cell in the centre
    occupations[centre], active_cell_counts[centre], age_of_first_occupations[centre], type_just_added[centre] = 1, 1, 0, 1


    no_cells = 1

    pops = [] #track number of cells
    overall_radii = [] #track the maximum radius

    while no_cells < max_cells and sum(active_cell_counts)>0:
        occupations, active_cell_counts, age_of_first_occupations = reproduce(occupations, active_cell_counts, age_of_first_occupations, t, alpha, dT, gamma, delta, beta, senesc_rate, prob_diff)
     #   print("reproduction")
        occupations, active_cell_counts, type_just_added, age_of_first_occupations = shift(occupations, active_cell_counts, age_of_first_occupations, type_just_added, t, alpha, dT, gamma, delta, beta, senesc_rate, prob_diff)
     #   print("shift")
        no_cells = sum(occupations)
        occupied = list(filter(lambda i: (occupations[i]==1), np.arange(N)))
        radii = [np.sqrt((xs[i]-xs[centre])**2 + (ys[i]-ys[centre])**2) for i in occupied]
        overall_radii.append(max(max(radii), r_0))
        print(no_cells)
        t += dT
        pops.append(no_cells)

    return occupations, t

In [5]:
def plot(name, occupations, elev, azim):
    to_plot = list(filter(lambda i: (occupations[i]==1), np.arange(N))) #find all occupied cells
    xs_to_plot, ys_to_plot, zs_to_plot = [xs[i] for i in to_plot], [ys[i] for i in to_plot], [zs[i] for i in to_plot]
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    p = ax.scatter(xs_to_plot, ys_to_plot, zs_to_plot)
    ax.view_init(elev=elev, azim=azim)
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    # make the grid lines transparent
    ax.xaxis._axinfo["grid"]['color'] = (1, 1, 1, 0)
    ax.yaxis._axinfo["grid"]['color'] = (1, 1, 1, 0)
    ax.zaxis._axinfo["grid"]['color'] = (1, 1, 1, 0)
    ax.set_axis_off()
    fig.tight_layout()
    fig.savefig(name+"_"+str(elev)+"_"+str(azim)+".png", format="png", dpi=100)
    #plt.show()
    plt.cla()
    plt.clf()
    plt.close('all')
    gc.collect()
    

In [6]:
def run_plot_save(name, alpha=1.0, dT=0.01, gamma=10, delta=1.0, beta=0.0, senesc_rate=0.0, prob_diff=1, max_cells=5000):
    occupations, t = run_simulation(alpha=alpha, dT=dT, gamma=gamma, delta=delta, beta=beta, senesc_rate=senesc_rate, prob_diff=prob_diff, max_cells=max_cells)
    np.save(name+"_occupations.npy", occupations)
    np.save(name+"_params_time.npy", [alpha, dT, gamma, delta, beta, senesc_rate, prob_diff, t])
    for elev in np.arange(-60, 60, 40):
        for azim in np.arange(-60, 60, 40): #cover all possible outlines, i.e. range over one half-surface
            plot(name, occupations, elev, azim)
    

In [7]:


def measure_curv(name, thresh_coeff=0.8):


    # Step 2: Load and preprocess the image
    image = io.imread(name)
    rgb_image = color.rgba2rgb(image)
    grayscale_image = color.rgb2gray(rgb_image)
    blurred_image = filters.gaussian(grayscale_image, sigma=10)
    threshold_value = thresh_coeff*filters.threshold_otsu(blurred_image)
    binary_image = grayscale_image > threshold_value


    # Step 3: Apply edge detection (optional)
    edges = feature.canny(binary_image)

    # Step 4: Find contours
    contours = measure.find_contours(edges, level=0.5)
    
    # Plot the original image
    #fig, ax = plt.subplots()
    #ax.imshow(rgb_image)

    # Plot contours over the original image
    #for contour in contours:
    #    ax.plot(contour[:, 1], contour[:, 0], linewidth=2, color='red')

    #largest_contour = max(contours, key=lambda x: len(x))
    #ax.plot(largest_contour[:, 1], largest_contour[:, 0], linewidth=2, color='yellow')

    # Concatenate all contours into a single array
    points = np.concatenate(contours)

    # Step 5: Compute concave hull
    multi_point = MultiPoint(points)
    hull = shapely.concave_hull(multi_point, ratio=0.05)


    # Plot the concave hull as a polygon
    ys, xs = hull.exterior.xy
    circularity = 4*np.pi*hull.area/(hull.length**2)
    #ax.plot(xs, ys, linewidth=2, color='green')

    #plt.show()
    del [image, rgb_image, grayscale_image, blurred_image, threshold_value, binary_image]
    print(circularity)
    return circularity


In [8]:
# beta=0.01
# max_cells=1000

# test = run_plot_save("depletion_test/"+str(beta)+"_"+str(max_cells), beta=beta, max_cells=max_cells)

In [9]:
# c_curv = np.zeros((16, 10, 3, 3))



# #first, vary c
# for c_, c in enumerate(range(10, 26)):
#     for r_, r in enumerate(range(10)):
#         for e_, elev in enumerate(np.arange(-60, 60, 40)):
#             for a_, azim in enumerate(np.arange(-60, 60, 40)):
#                 #run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_inhib_thresh=c)
#                 try:
#                     curv = measure_curv("c_varying/c_varying_"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     c_curv[c_][r_][e_][a_] = curv
#                     print(c, r, elev, azim)
#                 except:
#                     name = "c_varying/c_varying_"+str(c)+"_"+str(r)
#                     occupations = np.load("c_varying/c_varying_"+str(c)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
#                     plot(name, occupations, elev, azim)
#                     curv = measure_curv("c_varying/c_varying_"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     c_curv[c_][r_][e_][a_] = curv
#                 #if not ((e_ == 0) and (a_ == 0)): #keep 1 image
#                     #name = "c_varying/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png"
#                     #if os.path.exists(name):
#                         #os.remove(name)
#     print("Repeat " + str(r) + " completed.")


In [10]:
# #depletion test: vary beta
# for beta in [0.001*i for i in range(11)]:
#     for r in range(10):
#         run_plot_save("beta_varying_depletion/"+str(beta)+"_"+str(r), beta=beta, max_cells=5000)

In [9]:
#depletion test: vary beta
for beta in [0.009+0.0001*i for i in range(1, 10)]:
    for r in range(10):
        run_plot_save("beta_varying_depletion/"+str(beta)+"_"+str(r), beta=beta, max_cells=5000)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
3.0
3.0
3.0
3.0
3.0
3.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
7.0
7.0
7.0
7.0


KeyboardInterrupt: 

In [ ]:
#first, vary c
#for c in range(10, 26):
#    for r in range(10):
#        run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_inhib_thresh=c)
#        print("c_varying_"+str(c)+"_"+str(r))
#        assert(1==0)

# for alpha in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]:
#     for r in range(10):
#         run_plot_save("/HelenaCoggan/alpha_varying_"+str(alpha)+"_"+str(r), alpha=alpha, beta=10, contact_inhib_thresh=10)
#         print("alpha_varying_"+str(alpha)+"_"+str(r))
        
# for h in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
#     for r in range(10):
#         run_plot_save("/HelenaCoggan/h_varying_"+str(h)+"_"+str(r), beta=10, contact_inhib_thresh=10, h=h)
#         print("h_varying_"+str(h)+"_"+str(r))
        
# for tau in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
#     for r in range(10):
#         run_plot_save("/HelenaCoggan/tau_varying_"+str(tau)+"_"+str(r), beta=10, contact_inhib_thresh=10, delta=tau)
#         print("tau_varying_"+str(tau)+"_"+str(r))
        
# for sigma in [0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]:
#     for r in range(10):
#         run_plot_save("/HelenaCoggan/sigma_varying_"+str(sigma)+"_"+str(r), alpha=10, beta=0, contact_inhib_thresh=10, senesc_rate=sigma)
#         print("sigma_varying_"+str(sigma)+"_"+str(r))
        
        

In [12]:
beta_curv = np.zeros((11, 10, 3, 3))



#first, vary c
for c_, beta in enumerate([0.001*i for i in range(10)]+ [5000]):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_inhib_thresh=c)
                try:
                    curv = measure_curv("beta_varying_depletion/"+str(beta)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    beta_curv[c_][r_][e_][a_] = curv
                    print(beta, r, elev, azim)
                except:
                    name = "beta_varying_depletion/"+str(beta)+"_"+str(r)
                    occupations = np.load("beta_varying_depletion/"+str(beta)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("beta_varying_depletion/"+str(beta)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    beta_curv[c_][r_][e_][a_] = curv
                #if not ((e_ == 0) and (a_ == 0)): #keep 1 image
                    #name = "c_varying/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png"
                    #if os.path.exists(name):
                        #os.remove(name)
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_beta_depletion.npy", beta_curv)

0.9000246034030075
0.0 0 -60 -60
0.852662177343264
0.0 0 -60 -20
0.9138439337374787
0.0 0 -60 20
0.8629936684316608
0.0 0 -20 -60
0.8790251639214307
0.0 0 -20 -20
0.874866393612244
0.0 0 -20 20
0.869882244467985
0.0 0 20 -60
0.8649175208885519
0.0 0 20 -20
0.8779565557014803
0.0 0 20 20
0.8781697354712215
0.0 1 -60 -60
0.8612158649186313
0.0 1 -60 -20
0.8791522783448943
0.0 1 -60 20
0.8414919523403552
0.0 1 -20 -60
0.8596611955794939
0.0 1 -20 -20
0.8742048831843738
0.0 1 -20 20
0.8484927914868017
0.0 1 20 -60
0.889967073076687
0.0 1 20 -20
0.888799629100306
0.0 1 20 20
0.8686918570957961
0.0 2 -60 -60
0.8449576444352244
0.0 2 -60 -20
0.8474039652230151
0.0 2 -60 20
0.8302840038514988
0.0 2 -20 -60
0.8751530130962378
0.0 2 -20 -20
0.8724790405552266
0.0 2 -20 20
0.8258279453383283
0.0 2 20 -60
0.8801653427389553
0.0 2 20 -20
0.8901479520253183
0.0 2 20 20
0.9066673326510559
0.0 3 -60 -60
0.8812672386834458
0.0 3 -60 -20
0.8526229817003668
0.0 3 -60 20
0.8512525463570961
0.0 3 -20 -60
0

0.879590968422111
0.002 7 -60 20
0.8795912882691671
0.002 7 -20 -60
0.8616631677221677
0.002 7 -20 -20
0.8790648840355104
0.002 7 -20 20
0.8711800248121679
0.002 7 20 -60
0.8790899751869611
0.002 7 20 -20
0.8862113397086846
0.002 7 20 20
0.8779347146672576
0.002 8 -60 -60
0.8661895334222203
0.002 8 -60 -20
0.8626073464243599
0.002 8 -60 20
0.8635821219160429
0.002 8 -20 -60
0.87555865585381
0.002 8 -20 -20
0.8580871307140676
0.002 8 -20 20
0.869266716143564
0.002 8 20 -60
0.892305488952143
0.002 8 20 -20
0.876378340363193
0.002 8 20 20
0.8774592408375028
0.002 9 -60 -60
0.8801726820236355
0.002 9 -60 -20
0.8809626618808746
0.002 9 -60 20
0.8205922992791945
0.002 9 -20 -60
0.8824853297143047
0.002 9 -20 -20
0.8762331576861259
0.002 9 -20 20
0.8431163386738956
0.002 9 20 -60
0.8696349539568595
0.002 9 20 -20
0.8744425472657305
0.002 9 20 20
Repeat 9 completed.
0.8576382954511376
0.003 0 -60 -60
0.8765448722199136
0.003 0 -60 -20
0.8972108580483893
0.003 0 -60 20
0.8680884371954835
0.003 

0.8654479633451253
0.005 3 20 -60
0.8790522918529128
0.005 3 20 -20
0.877295983471652
0.005 3 20 20
0.885529745136138
0.005 4 -60 -60
0.8820974167445418
0.005 4 -60 -20
0.8913158717685458
0.005 4 -60 20
0.8540698758788055
0.005 4 -20 -60
0.849877048215847
0.005 4 -20 -20
0.8579007934259624
0.005 4 -20 20
0.8710773728216833
0.005 4 20 -60
0.8627629210351042
0.005 4 20 -20
0.8810035360173106
0.005 4 20 20
0.8879233866321543
0.005 5 -60 -60
0.9018097728666415
0.005 5 -60 -20
0.915733139421162
0.005 5 -60 20
0.8884114448658329
0.005 5 -20 -60
0.86727531051232
0.005 5 -20 -20
0.8726195791937559
0.005 5 -20 20
0.8659824247041747
0.005 5 20 -60
0.8630678020903306
0.005 5 20 -20
0.8650490799238475
0.005 5 20 20
0.880878999220392
0.005 6 -60 -60
0.887005415642846
0.005 6 -60 -20
0.887520437012011
0.005 6 -60 20
0.8641972620886763
0.005 6 -20 -60
0.8683803317284183
0.005 6 -20 -20
0.8857405455672998
0.005 6 -20 20
0.8603202928581278
0.005 6 20 -60
0.8684248457823939
0.005 6 20 -20
0.878097730905

0.8677840073021537
0.008 0 -60 20
0.8208654580070424
0.008 0 -20 -60
0.8677212616995289
0.008 0 -20 -20
0.8518614344946621
0.008 0 -20 20
0.8637311758991612
0.008 0 20 -60
0.8493278095541653
0.008 0 20 -20
0.8734679912806183
0.008 0 20 20
0.8879933509620659
0.008 1 -60 -60
0.8927143033305913
0.008 1 -60 -20
0.8631743292239529
0.008 1 -60 20
0.8463244685571719
0.008 1 -20 -60
0.8583300497227174
0.008 1 -20 -20
0.8595672854894014
0.008 1 -20 20
0.8675451158404369
0.008 1 20 -60
0.8582623461215757
0.008 1 20 -20
0.8947242767443357
0.008 1 20 20
0.8576880693957519
0.008 2 -60 -60
0.8391632905440501
0.008 2 -60 -20
0.8792439498055287
0.008 2 -60 20
0.8346711784884511
0.008 2 -20 -60
0.8422702828205103
0.008 2 -20 -20
0.8471836632477074
0.008 2 -20 20
0.8400335759017956
0.008 2 20 -60
0.8567486953341824
0.008 2 20 -20
0.8613817701744454
0.008 2 20 20
0.8625453000872929
0.008 3 -60 -60
0.8927196166573373
0.008 3 -60 -20
0.8771125089907438
0.008 3 -60 20
0.852382560459985
0.008 3 -20 -60
0.848

0.6962678690703542
5000 2 -20 20
0.7384433700877012
5000 2 20 -60
0.7667393225598915
5000 2 20 -20
0.6815771451572513
5000 2 20 20
0.623894647529949
5000 3 -60 -60
0.6065713785219805
5000 3 -60 -20
0.6055980467722679
5000 3 -60 20
0.7518297149631589
5000 3 -20 -60
0.7626825780091999
5000 3 -20 -20
0.6523060948951148
5000 3 -20 20
0.7662537764876528
5000 3 20 -60
0.7256573156500651
5000 3 20 -20
0.6423220119542568
5000 3 20 20
0.676369995291522
5000 4 -60 -60
0.6733578129671439
5000 4 -60 -20
0.6819737922186023
5000 4 -60 20
0.6607823175253352
5000 4 -20 -60
0.6829517040514056
5000 4 -20 -20
0.7360390323381527
5000 4 -20 20
0.6507310614123257
5000 4 20 -60
0.6474364227734168
5000 4 20 -20
0.6925536670701001
5000 4 20 20
0.7119678305730252
5000 5 -60 -60
0.7030524262567287
5000 5 -60 -20
0.6702081808998154
5000 5 -60 20
0.7057344132846556
5000 5 -20 -60
0.6572255354844586
5000 5 -20 -20
0.6292649717406871
5000 5 -20 20
0.7046205697878114
5000 5 20 -60
0.7214002745968187
5000 5 20 -20
0.6

In [13]:
max_cells_curv = np.zeros((11, 10, 3, 3))



#first, vary c
for c_, max_cells in enumerate([100, 200, 500, 1000, 2000, 3000, 4000, 5000]):
    for r_, r in enumerate(range(10)):
        for e_, elev in enumerate(np.arange(-60, 60, 40)):
            for a_, azim in enumerate(np.arange(-60, 60, 40)):
                #run_plot_save("c_varying/"+str(c)+"_"+str(r), max_cells=10, contact_inhib_thresh=c)
                try:
                    curv = measure_curv("cells_varying_depletion/"+str(max_cells)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    max_cells_curv[c_][r_][e_][a_] = curv
                    print(max_cells, r, elev, azim)
                except:
                    name = "cells_varying_depletion/"+str(max_cells)+"_"+str(r)
                    occupations = np.load("cells_varying_depletion/"+str(max_cells)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
                    plot(name, occupations, elev, azim)
                    curv = measure_curv("cells_varying_depletion/"+str(max_cells)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
                    max_cells_curv[c_][r_][e_][a_] = curv
                #if not ((e_ == 0) and (a_ == 0)): #keep 1 image
                    #name = "c_varying/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png"
                    #if os.path.exists(name):
                        #os.remove(name)
    print("Repeat " + str(r) + " completed.")

    
np.save("curvature_matrix_max_cells_depletion.npy", max_cells_curv)

0.004054958301456172
100 0 -60 -60
0.004360002401440631
100 0 -60 -20
0.00415524394001503
100 0 -60 20
0.005059354660595564
100 0 -20 -60
0.0042689795477659186
100 0 -20 -20
0.004425816099369299
100 0 -20 20
0.0044763345494528
100 0 20 -60
0.004428201786651241
100 0 20 -20
0.004396329584918366
100 0 20 20
0.0034888229994153577
100 1 -60 -60
0.0036119638920378565
100 1 -60 -20
0.003565713021797624
100 1 -60 20
0.004970304286807858
100 1 -20 -60
0.005222240275105227
100 1 -20 -20
0.005437480014497882
100 1 -20 20
0.005160753342257828
100 1 20 -60
0.004941835424580801
100 1 20 -20
0.004911200882755261
100 1 20 20
0.005173990480494658
100 2 -60 -60
0.005279117936051326
100 2 -60 -20
0.0051908192735458745
100 2 -60 20
0.003702958092009713
100 2 -20 -60
0.0042638476620147445
100 2 -20 -20
0.00457189644279442
100 2 -20 20
0.0058962262285796275
100 2 20 -60
0.004507587147459269
100 2 20 -20
0.005072518501413191
100 2 20 20
0.0035916909287944695
100 3 -60 -60
0.0036334581129485307
100 3 -60 -20

0.3096832090249757
500 6 -20 20
0.014415921073130354
500 6 20 -60
0.09347712621008886
500 6 20 -20
0.2216447431822809
500 6 20 20
0.16351370759724426
500 7 -60 -60
0.1629953907765907
500 7 -60 -20
0.1855761444245995
500 7 -60 20
0.005810430619868273
500 7 -20 -60
0.016794571864231125
500 7 -20 -20
0.36258580791996303
500 7 -20 20
0.005697343425371145
500 7 20 -60
0.018142137725952533
500 7 20 -20
0.041756592237067215
500 7 20 20
0.05211569267869836
500 8 -60 -60
0.08903398562261604
500 8 -60 -20
0.10206412749970571
500 8 -60 20
0.011533624711886755
500 8 -20 -60
0.12821405575677364
500 8 -20 -20
0.18166942311685322
500 8 -20 20
0.0111940324547902
500 8 20 -60
0.22710089458932145
500 8 20 -20
0.2012526115800765
500 8 20 20
0.011693215844621673
500 9 -60 -60
0.3348646263084157
500 9 -60 -20
0.1943523884488177
500 9 -60 20
0.005174127525442392
500 9 -20 -60
0.060232569150044435
500 9 -20 -20
0.07928449983315415
500 9 -20 20
0.006027104033071909
500 9 20 -60
0.009174557154083196
500 9 20 -

0.7851107587724396
3000 3 -20 20
0.7384138398644547
3000 3 20 -60
0.8207817485106987
3000 3 20 -20
0.8073069897686742
3000 3 20 20
0.7711044456732425
3000 4 -60 -60
0.7574800951587992
3000 4 -60 -20
0.7004894121191614
3000 4 -60 20
0.6107968382801524
3000 4 -20 -60
0.7587086766114091
3000 4 -20 -20
0.7897865651653928
3000 4 -20 20
0.634734048211555
3000 4 20 -60
0.7810063159308093
3000 4 20 -20
0.8395634100598603
3000 4 20 20
0.7514578301025922
3000 5 -60 -60
0.7126092916661554
3000 5 -60 -20
0.6419900011163397
3000 5 -60 20
0.7283843331123999
3000 5 -20 -60
0.8120843432884193
3000 5 -20 -20
0.7461489739914776
3000 5 -20 20
0.7660013296847825
3000 5 20 -60
0.7814275551794906
3000 5 20 -20
0.7597825710700161
3000 5 20 20
0.8096997572417203
3000 6 -60 -60
0.7377281516269397
3000 6 -60 -20
0.7106387433374024
3000 6 -60 20
0.7858212511461171
3000 6 -20 -60
0.7785784091935858
3000 6 -20 -20
0.8005948259063829
3000 6 -20 20
0.7871229223759318
3000 6 20 -60
0.7758824815363529
3000 6 20 -20
0.

In [ ]:
# h_curv = np.zeros((10, 10, 3, 3))



# #first, vary c
# for c_, h in enumerate([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]):
#     for r_, r in enumerate(range(10)):
#         for e_, elev in enumerate(np.arange(-60, 60, 40)):
#             for a_, azim in enumerate(np.arange(-60, 60, 40)):
#                 #run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_inhib_thresh=c)
#                 try:
#                     curv = measure_curv("h_varying/h_varying_"+str(h)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     h_curv[c_][r_][e_][a_] = curv
#                     print(h, r, elev, azim)
#                 except:
#                     name = "h_varying/h_varying_"+str(h)+"_"+str(r)
#                     occupations = np.load("h_varying/h_varying_"+str(h)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
#                     plot(name, occupations, elev, azim)
#                     curv = measure_curv("h_varying/h_varying_"+str(h)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     h_curv[c_][r_][e_][a_] = curv
#                 #if not ((e_ == 0) and (a_ == 0)): #keep 1 image
#                     #name = "c_varying/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png"
#                     #if os.path.exists(name):
#                         #os.remove(name)
#     print("Repeat " + str(r) + " completed.")

    
# np.save("curvature_matrix_h.npy", h_curv)

In [ ]:
# tau_curv = np.zeros((10, 10, 3, 3))



# #first, vary c
# for c_, tau in enumerate([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]):
#     for r_, r in enumerate(range(10)):
#         for e_, elev in enumerate(np.arange(-60, 60, 40)):
#             for a_, azim in enumerate(np.arange(-60, 60, 40)):
#                 #run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_intauib_ttaurestau=c)
#                 try:
#                     curv = measure_curv("tau_varying/tau_varying_"+str(tau)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     tau_curv[c_][r_][e_][a_] = curv
#                     print(tau, r, elev, azim)
#                 except:
#                     name = "tau_varying/tau_varying_"+str(tau)+"_"+str(r)
#                     occupations = np.load("tau_varying/tau_varying_"+str(tau)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
#                     plot(name, occupations, elev, azim)
#                     curv = measure_curv("tau_varying/tau_varying_"+str(tau)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     tau_curv[c_][r_][e_][a_] = curv
#                 #if not ((e_ == 0) and (a_ == 0)): #keep 1 image
#                     #name = "c_varying/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png"
#                     #if os.pattau.exists(name):
#                         #os.remove(name)
#     print("Repeat " + str(r) + " completed.")

    
# np.save("curvature_matrix_tau.npy", tau_curv)

In [ ]:
# sigma_curv = np.zeros((10, 10, 3, 3))



# #first, vary c
# for c_, sigma in enumerate([0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]):
#     for r_, r in enumerate(range(10)):
#         for e_, elev in enumerate(np.arange(-60, 60, 40)):
#             for a_, azim in enumerate(np.arange(-60, 60, 40)):
#                 #run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_insigmaib_tsigmaressigma=c)
#                 try:
#                     curv = measure_curv("sigma_varying/sigma_varying_"+str(sigma)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     sigma_curv[c_][r_][e_][a_] = curv
#                     print(sigma, r, elev, azim)
#                 except:
#                     name = "sigma_varying/sigma_varying_"+str(sigma)+"_"+str(r)
#                     occupations = np.load("sigma_varying/sigma_varying_"+str(sigma)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
#                     plot(name, occupations, elev, azim)
#                     curv = measure_curv("sigma_varying/sigma_varying_"+str(sigma)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     sigma_curv[c_][r_][e_][a_] = curv
#                 #if not ((e_ == 0) and (a_ == 0)): #keep 1 image
#                     #name = "c_varying/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png"
#                     #if os.patsigma.exists(name):
#                         #os.remove(name)
#     print("Repeat " + str(r) + " completed.")

    
# np.save("curvature_matrix_sigma.npy", sigma_curv)

In [ ]:
# #now generate figures for fig3- this time only the necessary ones
# for beta in range(1, 11):
#     for r in range(10):
#         run_plot_save("beta_varying_fig3/"+str(beta)+"_"+str(r), alpha=1, beta=beta, contact_inhib_thresh=20, h=1, delta=1, max_cells=10000)
#         print("beta_varying_fig3_"+str(beta)+"_"+str(r))

# for h in list(np.power(10, np.arange(-5, 5))):
#     for r in range(10):
#         run_plot_save("h_varying_fig3/"+str(h)+"_"+str(r), alpha=1, beta=2, contact_inhib_thresh=20, h=h, delta=1, max_cells=10000)
#         print("h_varying_fig3_"+str(h)+"_"+str(r))


# for tau in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
#     for r in range(10):
#         run_plot_save("tau_varying_fig3/"+str(tau)+"_"+str(r), alpha=1, beta=2, contact_inhib_thresh=20, h=1, delta=tau, max_cells=10000)
#         print("tau_varying_fig3_"+str(tau)+"_"+str(r))


# for c in range(10, 26):
#     for r in range(10):
#         run_plot_save("c_varying_fig3/"+str(c)+"_"+str(r), alpha=1, beta=2, contact_inhib_thresh=c, h=1, delta=1, max_cells=10000)
#         print("c_varying_fig3_"+str(c)+"_"+str(r))


In [ ]:
# beta_curv_fig3 = np.zeros((10, 10, 3, 3))



# #first, vary c
# for c_, beta in enumerate(range(1, 11)):
#     for r_, r in enumerate(range(10)):
#         for e_, elev in enumerate(np.arange(-60, 60, 40)):
#             for a_, azim in enumerate(np.arange(-60, 60, 40)):
#                 #run_plot_save("c_varying/"+str(c)+"_"+str(r), beta=10, contact_inbetaib_tbetaresbeta=c)
#                 try:
#                     curv = measure_curv("beta_varying_fig3/"+str(beta)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     beta_curv_fig3[c_][r_][e_][a_] = curv
#                     print(beta, r, elev, azim)
#                 except:
#                     name = "beta_varying_fig3/"+str(beta)+"_"+str(r)
#                     occupations = np.load("beta_varying_fig3/"+str(beta)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
#                     plot(name, occupations, elev, azim)
#                     curv = measure_curv("beta_varying_fig3/"+str(beta)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     beta_curv_fig3[c_][r_][e_][a_] = curv
#                 #if not ((e_ == 0) and (a_ == 0)): #keep 1 image
#                     #name = "c_varying/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png"
#                     #if os.patbeta.exists(name):
#                         #os.remove(name)
#     print("Repeat " + str(r) + " completed.")

    
# np.save("curvature_matrix_beta_fig3.npy", beta_curv_fig3)

In [ ]:
# h_curv_fig3 = np.zeros((10, 10, 3, 3))



# #first, vary c
# for c_, h in enumerate([0.01, 0.1, 1.0, 10, 100]):
#     for r_, r in enumerate(range(10)):
#          for e_, elev in enumerate(np.arange(-60, 60, 40)):
#             for a_, azim in enumerate(np.arange(-60, 60, 40)):
#                 try:
#                     curv = measure_curv("h_varying_fig3/"+str(h)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     h_curv_fig3[c_][r_][e_][a_] = curv
#                     print(h, r, elev, azim)
#                 except:
#                     name = "h_varying_fig3/"+str(h)+"_"+str(r)
#                     occupations = np.load("h_varying_fig3/"+str(h)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
#                     plot(name, occupations, elev, azim)
#                     curv = measure_curv("h_varying_fig3/"+str(h)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     h_curv_fig3[c_][r_][e_][a_] = curv
#     print("Repeat " + str(r) + " completed.")

    
# np.save("curvature_matrix_h_fig3.npy", h_curv_fig3)

In [ ]:
# tau_curv_fig3 = np.zeros((10, 10, 3, 3))



# #first, vary c
# for c_, tau in enumerate([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]):
#     for r_, r in enumerate(range(10)):
#         for e_, elev in enumerate(np.arange(-60, 60, 40)):
#             for a_, azim in enumerate(np.arange(-60, 60, 40)):
#                 #run_plot_save("c_varying/"+str(c)+"_"+str(r), tau=10, contact_intauib_ttaurestau=c)
#                 try:
#                     curv = measure_curv("tau_varying_fig3/"+str(tau)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     tau_curv_fig3[c_][r_][e_][a_] = curv
#                     print(tau, r, elev, azim)
#                 except:
#                     name = "tau_varying_fig3/"+str(tau)+"_"+str(r)
#                     occupations = np.load("tau_varying_fig3/"+str(tau)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
#                     plot(name, occupations, elev, azim)
#                     curv = measure_curv("tau_varying_fig3/"+str(tau)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     tau_curv_fig3[c_][r_][e_][a_] = curv
#     print("Repeat " + str(r) + " completed.")

    
# np.save("curvature_matrix_tau_fig3.npy", tau_curv_fig3)

In [ ]:
# c_curv_fig3 = np.zeros((16, 10, 3, 3))



# #first, vary c
# for c_, c in enumerate(range(10, 26)):
#     for r_, r in enumerate(range(10)):
#         for e_, elev in enumerate(np.arange(-60, 60, 40)):
#             for a_, azim in enumerate(np.arange(-60, 60, 40)):
#                 #run_plot_save("c_varying/"+str(c)+"_"+str(r), c=10, contact_incib_tcresc=c)
#                 try:
#                     curv = measure_curv("c_varying_fig3/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     c_curv_fig3[c_][r_][e_][a_] = curv
#                     print(c, r, elev, azim)
#                 except:
#                     name = "c_varying_fig3/"+str(c)+"_"+str(r)
#                     occupations = np.load("c_varying_fig3/"+str(c)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
#                     plot(name, occupations, elev, azim)
#                     curv = measure_curv("c_varying_fig3/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     c_curv_fig3[c_][r_][e_][a_] = curv
#     print("Repeat " + str(r) + " completed.")

    
# np.save("curvature_matrix_c_fig3.npy", c_curv_fig3)

In [ ]:
# alpha_curv_fig4 = np.zeros((10, 10, 3, 3))



# #first, vary c
# for c_, alpha in enumerate([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]):
#     for r_, r in enumerate(range(10)):
#         for e_, elev in enumerate(np.arange(-60, 60, 40)):
#             for a_, azim in enumerate(np.arange(-60, 60, 40)):
#                 #run_plot_save("c_varying/"+str(c)+"_"+str(r), alpha=10, contact_inalphaib_talpharesalpha=c)
#                 try:
#                     curv = measure_curv("alpha_varying_fig4/"+str(alpha)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     alpha_curv_fig4[c_][r_][e_][a_] = curv
#                     print(alpha, r, elev, azim)
#                 except:
#                     name = "alpha_varying_fig4/"+str(alpha)+"_"+str(r)
#                     occupations = np.load("alpha_varying_fig4/"+str(alpha)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
#                     plot(name, occupations, elev, azim)
#                     curv = measure_curv("alpha_varying_fig4/"+str(alpha)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     alpha_curv_fig4[c_][r_][e_][a_] = curv
#     print("Repeat " + str(r) + " completed.")

    
# np.save("curvature_matrix_alpha_fig4.npy", alpha_curv_fig4)

In [ ]:
# c_curv_fig4 = np.zeros((16, 10, 3, 3))



# #first, vary c
# for c_, c in enumerate(range(10, 26)):
#     for r_, r in enumerate(range(10)):
#         for e_, elev in enumerate(np.arange(-60, 60, 40)):
#             for a_, azim in enumerate(np.arange(-60, 60, 40)):
#                 #run_plot_save("c_varying/"+str(c)+"_"+str(r), c=10, contact_incib_tcresc=c)
#                 try:
#                     curv = measure_curv("c_varying_fig4/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     c_curv_fig4[c_][r_][e_][a_] = curv
#                     print(c, r, elev, azim)
#                 except:
#                     name = "c_varying_fig4/"+str(c)+"_"+str(r)
#                     occupations = np.load("c_varying_fig4/"+str(c)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
#                     plot(name, occupations, elev, azim)
#                     curv = measure_curv("c_varying_fig4/"+str(c)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     c_curv_fig4[c_][r_][e_][a_] = curv
#     print("Repeat " + str(r) + " completed.")

    
# np.save("curvature_matrix_c_fig4.npy", c_curv_fig4)

In [ ]:
# q_curv_fig5 = np.zeros((10, 10, 3, 3))



# #first, vary c
# for c_, q in enumerate([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
#     for r_, r in enumerate(range(10)):
#         for e_, elev in enumerate(np.arange(-60, 60, 40)):
#             for a_, azim in enumerate(np.arange(-60, 60, 40)):
#                 #run_plot_save("c_varying/"+str(c)+"_"+str(r), q=10, contact_inqib_tqresq=c)
#                 try:
#                     curv = measure_curv("q_varying_fig5/"+str(q)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     q_curv_fig5[c_][r_][e_][a_] = curv
#                     print(q, r, elev, azim)
#                 except:
#                     name = "q_varying_fig5/"+str(q)+"_"+str(r)
#                     occupations = np.load("q_varying_fig5/"+str(q)+"_"+str(r)+"_occupations.npy", allow_pickle=True)
#                     plot(name, occupations, elev, azim)
#                     curv = measure_curv("q_varying_fig5/"+str(q)+"_"+str(r)+"_"+str(elev)+"_"+str(azim)+".png")
#                     q_curv_fig5[c_][r_][e_][a_] = curv
#     print("Repeat " + str(r) + " completed.")

    
# np.save("curvature_matrix_q_fig5.npy", q_curv_fig5)